In [11]:
# setup cell
import time
import uuid
import json
import os
import requests
import random
import gzip
import calendar
from datetime import datetime
import pandas as pd
import urllib.request
from urllib.parse import urlparse
import re
import html

## Extract data from HTML from sqlite db

In [12]:
import sqlite3
from db import scan_table_limit_offset, decompress_string
from bs4 import BeautifulSoup

db_conn = sqlite3.connect(os.path.join(os.path.dirname(__name__), "sample_data/films.db"))

def get_film_id_from_url(url):
    film_id = url.split('/')[4]
    assert film_id[0:2] == 'tt'
    return film_id

In [13]:
import logging

def setup_logger():
    # Create logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_folder = os.path.join(os.path.dirname(__name__), "logs")
    if not os.path.isdir(log_folder):
        os.mkdir(log_folder)
        
    log_file = os.path.join(log_folder, f"s_log_{current_time}.txt")

    # Create file handler
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)

    # Create formatter and add it to the handler
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)

    # Add the file handler to the logger
    logger.addHandler(file_handler)

    return logger

logger = setup_logger()

Generate keyword CSV

In [14]:
all_dfs = []

# scan pages
def process_keywords(df):
    
    for index, row in df.iterrows():
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text of all elements with class="paragraph"
        items = soup.find_all(class_="ipc-metadata-list-summary-item__tc")
        keywords = [p.text.strip() for p in items]
        if len(keywords) == 0:
            logger.debug('No keywords found '+film_id)
            if "have any plot keywords for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                keywords.append('None')
        keywords_df = pd.DataFrame(keywords, columns=['keyword'])
        keywords_df['film_id'] = film_id
        
        all_dfs.append(keywords_df)


logger.info(">>> Scraping Keywords ")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%keywords%'", 1000, process_keywords)
all_keywords_df = pd.concat(all_dfs)
all_keywords_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/film_keywords.csv"), index=False)
all_keywords_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/film_keywords.json"), orient="records")
print("ok")
del all_dfs

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok


In [ ]:
Generate plot summaries

In [9]:
all_dfs_summary = []

def clean_summary(s):
    s = s.replace('\t', '    ').strip()
    return s

# scan pages
def process_plots(df):
    for index, row in df.iterrows():
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text of all elements with class="paragraph"_counter
        sectionA = soup.find_all('section', class_='ipc-page-section ipc-page-section--base')
        for sectB in sectionA:
            containerA = sectB.find('div', class_="ipc-title ipc-title--base ipc-title--section-title ipc-title--on-textPrimary")

            # check if page is valid
            if containerA:
                titleA = containerA.find('span').get_text()

                # Extract Summaries
                if titleA == 'Summaries':
                    boxes = sectB.find_all(class_="ipc-html-content ipc-html-content--base")

                    # Check if summaries section has valid data
                    if boxes:
                        item_len = 0
                        for box in boxes:
                            item_len += 1
                            items = box.find_all(class_="ipc-html-content-inner-div")
                            item_texts = [p.text.strip() for p in items]
                            item_texts = [clean_summary(p) for p in item_texts]
                            if len(item_texts) == 0:
                                logger.debug('No summaries found '+film_id)
                                if "have any plot for this title yet" in html.unescape(html_content):
                                    logger.debug('Page content not valid '+film_id)
                                    item_texts.append('None')
                            item_df = pd.DataFrame(item_texts, columns=['summary'])
                            item_df['film_id'] = film_id
                            item_df['summary_id'] = item_len
                            all_dfs_summary.append(item_df)
                            
                    else:    # If summaries section is not valid enter empty row for the film
                        logger.debug('No summaries found '+film_id)
                        if "have any plot for this title yet" in html.unescape(html_content):
                            logger.debug('Page content not valid '+film_id)
                        item_texts=['None']
                        item_df = pd.DataFrame(item_texts, columns=['summary'])
                        item_df['film_id'] = film_id
                        item_df['summary_id'] = 1
                        all_dfs_summary.append(item_df)                       
            
            else:
                # if page is not valid, enter empty row for summary and synopsis for the film
                logger.debug('No plots found '+film_id)
                if "have any plot for this title yet" in html.unescape(html_content):
                    logger.debug('Page content not valid '+film_id)
                item_texts=['None']
                item_df = pd.DataFrame(item_texts, columns=['summary'])
                item_df['film_id'] = film_id
                item_df['summary_id'] = 1
                all_dfs_summary.append(item_df)


logger.info(">>> Scraping Plot Summary")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%plotsummary%'", 1000, process_plots)
all_plots_df = pd.concat(all_dfs_summary)
all_plots_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/film_plots.tsv"), sep='\t', index=False)
all_plots_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/film_plots.json"), orient="records")
print("ok")

del all_dfs_summary

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok


Generate synopsis

In [10]:
all_dfs_synopsis = []

def clean_summary(s):
    s = s.replace('\t', '    ').strip()
    return s

# scan pages
def process_synopsis(df):
    for index, row in df.iterrows():
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text of all elements with class="paragraph"_counter
        sectionA = soup.find_all('section', class_='ipc-page-section ipc-page-section--base')
        for sectB in sectionA:
            containerA = sectB.find('div', class_="ipc-title ipc-title--base ipc-title--section-title ipc-title--on-textPrimary")

            # check if page is valid
            if containerA:
                titA = containerA.find('span').get_text()

                if titA == 'Synopsis':
                    boxes = sectB.find_all(class_="ipc-html-content ipc-html-content--base")
                    # Check if synopsis section has valid data
                    if boxes:
                        for box in boxes:
                            items = box.find_all(class_="ipc-html-content-inner-div")
                            item_texts = [p.text.strip() for p in items]
                            item_texts = [clean_summary(p) for p in item_texts]
                            if len(item_texts) == 0:
                                logger.debug('No synopsis found '+film_id)
                                if "have any plot for this title yet" in html.unescape(html_content):
                                    #logger.debug('Page content not valid '+film_id)
                                    item_texts.append('None')
                            item_df = pd.DataFrame(item_texts, columns=['synopsis'])
                            item_df['film_id'] = film_id
                            all_dfs_synopsis.append(item_df)
                    else:
                        # If synopsis section is not valid enter empty row for the film
                        logger.debug('No synopsis found '+film_id)
                        if "have any synopsis for this title yet" in html.unescape(html_content):
                            logger.debug('Page content not valid '+film_id)
                        item_texts=['None']
                        item_df = pd.DataFrame(item_texts, columns=['synopsis'])
                        item_df['film_id'] = film_id
                        all_dfs_synopsis.append(item_df)
                        
            else:
                # if page is not valid, enter empty row for summary and synopsis for the film
                logger.debug('No plots found '+film_id)
                if "have any plot for this title yet" in html.unescape(html_content):
                    logger.debug('Page content not valid '+film_id)
                item_texts=['None']
                item_df = pd.DataFrame(item_texts, columns=['synopsis'])
                item_df['film_id'] = film_id
                all_dfs_synopsis.append(item_df)

logger.info(">>> Scraping Synopsis")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%plotsummary%'", 1000, process_synopsis)

all_plots_df_syp =  pd.concat(all_dfs_synopsis)
all_plots_df_syp.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/film_synopsis.tsv"), sep='\t', index=False)
all_plots_df_syp.to_json(os.path.join(os.path.dirname(__name__), "sample_data/film_synopsis.json"), orient="records")
print("ok")

del all_dfs_synopsis

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok


Generate Location CSV

In [15]:
all_locations = []

#scan pages
def process_locations(df):
    for index, row in df.iterrows():
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text of all elements with class="paragraph"
        items = soup.find_all(class_="sc-bec740f7-5 eLQUFg")
        locations = [p.text.strip() for p in items]
        
        if len(locations) == 0:
            logger.debug('No locations found '+film_id)
            if "have any filming & production for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                locations.append('None')
        locations_df = pd.DataFrame(locations, columns=['locations'])
        locations_df['film_id'] = film_id
        
        all_locations.append(locations_df)

logger.info(">>> Scraping locations")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%locations%'", 1000, process_locations)
all_locations_df = pd.concat(all_locations)
all_locations_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/filming_locations.csv"), index=False)
all_locations_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/filming_locations.json"), orient="records")
print("ok")

del all_locations

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok


Generate User Reviews TSV

In [16]:
all_dfs = []

def clean_summary(s):
    s = s.replace('\t', '    ').strip()
    return s

# scan pages
def process_user_reviews(df):
    for index, row in df.iterrows():
        all_titles = []
        all_reviews = []
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract text and title elements of all reviews
        box = soup.find('div', class_='lister-list')
        # check if page information is valid or not
        if box:
            items = box.findAll(class_="lister-item")
            if items:
                for item in items:
                    title = item.find(class_="title").get_text()
                    if title:
                        all_titles.append(title)
                    else:
                        all_titles.append('None')
                    review = item.find(class_="text").get_text(separator=' ', strip=True)
                    if review:
                        all_reviews.append(review)
                    else:
                        all_reviews.append('None')
            else:
                logger.debug('No user reviews found '+film_id)
                all_titles.append('None')
                all_reviews.append('None')    
        else:
            logger.debug('No user reviews found '+film_id)
            if "have any user reviews for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                all_titles.append('None')
                all_reviews.append('None')    
               
        my_dict = {'title':all_titles, 'user_review':all_reviews}
        item_df = pd.DataFrame.from_dict(my_dict)
        item_df['film_id'] = film_id
        item_df['review_id'] = range(len(item_df))
        item_df['review_id'] = item_df['review_id']+1
        all_dfs.append(item_df)

logger.info(">>> Scraping user reviews")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%reviews?sort=curated%'", 1000, process_user_reviews)
all_plots_df = pd.concat(all_dfs)
all_plots_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/user_reviews.tsv"), sep='\t', index=False)
print("ok")
all_plots_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/user_reviews.json"), orient="records")
print("ok")

del all_dfs

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok
ok


Generate Critic Reviews TSV

In [17]:
all_dfs = []

# scan pages
def process_critics_reviws(df):
    #reviews_counter = 0
    for index, row in df.iterrows():
        all_scores=[]
        all_titles = []
        all_critics=[]
        all_reviews = []
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract text and title elements of all reviews
        box = soup.find('div', class_='sc-f65f65be-0 bBlII')

        # check if page information is valid or not
        if not box:
            logger.debug('No critic reviews found '+film_id)
            if "have any metacritic reviews for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                all_scores.append('None')
                all_titles.append('None')
                all_critics.append('None')
                all_reviews.append('None') 
        else:
            items = box.find_all('li',class_="ipc-metadata-list__item")
            for item in items:
                score=item.find('div',class_='sc-d8486f96-2').get_text()
                all_scores.append(score)
                titles = item.find('div', class_='sc-d8486f96-3 bvYFxQ')
                title = titles.find('span').get_text()
                if not title:
                    title = titles.find('a').get_text()
                    if not title:
                        title = 'None'
                all_titles.append(title)
                critics=item.find_all('span', class_='sc-d8486f96-6 ifLlre')
                if critics:
                    critic=item.find('span', class_='sc-d8486f96-6 ifLlre').get_text()
                else: 
                    critics=item.find_all('a')[1]
                    if critics:
                        for critic in critics:
                            critic=item.find('a').get_text()
                    else:
                        critic=None
                all_critics.append(critic)
                reviews = item.find_all('div')
                if reviews:
                    review = reviews[-1].get_text()
                else:
                    review=None
                all_reviews.append(review)
    
        my_dict = {'score':all_scores,'title':all_titles, 'critic_name':all_critics,'critic_review':all_reviews}    
        if len(my_dict) == 0:
            logger.debug('No critic reviews found '+film_id)
            if "have any metacritic reviews for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                all_scores.append('None')
                all_titles.append('None')
                all_critics.append('None')
                all_reviews.append('None')
        
        item_df = pd.DataFrame.from_dict(my_dict)
        item_df['film_id'] = film_id
        item_df['c_review_id'] = range(len(item_df))
        item_df['c_review_id'] = item_df['c_review_id']+1
        all_dfs.append(item_df)
  
logger.info(">>> Scraping critic reviews")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%criticreviews%'", 1000, process_critics_reviws)
all_criticreviews_df = pd.concat(all_dfs)
all_criticreviews_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data\critic_reviews.tsv"), sep='\t', index=False)
all_criticreviews_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/critic_reviews.json"), orient="records")
print("-ok")
del all_dfs

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
-ok


Generate base details CSV

In [8]:
#work on this
all_dfs = []

# scan pages
def process_details(df):
    for index, row in df.iterrows():
        all_detail_item=[]
        all_detail_result = []
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract items of all details
        items=soup.find('div',class_='sc-978e9339-1 ihWZgK ipc-page-grid__item ipc-page-grid__item--span-2')
        if items:
            items1 = items.find_all('section')
            if len(items1) == 0:
                assert "have any detail for this title yet" in html_content.lower()
                continue
     
            for item in items1:
                texts=item.find('span')
                if texts:
                    texts=item.find('span').get_text()
                    if texts=="Details":
                        rows = item.find_all('li',class_='ipc-metadata-list__item')[:-1]
                        for row in rows:
                            detail_item=row.find('a',class_='ipc-metadata-list-item__label')
                            if detail_item:
                                detail_item=row.find('a',class_='ipc-metadata-list-item__label').get_text()
                            else:
                                detail_item=row.find('span',class_='ipc-metadata-list-item__label').get_text()
    
                            result_lists = row.find_all('li', class_='ipc-inline-list__item')
                            for result_list in result_lists:
                                detail_result = result_list.find('a', class_='ipc-metadata-list-item__list-content-item')
                                if detail_result:
                                    detail_result = result_list.find('a', class_='ipc-metadata-list-item__list-content-item').get_text()
                                else:
                                    detail_result = result_list.find('span', class_='ipc-metadata-list-item__list-content-item').get_text()
                                all_detail_result.append(detail_result)
                                all_detail_item.append(detail_item)
                                
        else:
            logger.debug('No base details found '+film_id)
            if "have any details for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                all_detail_result.append('None')
                all_detail_item.append('None')                             
    
        my_dict = {'detail_item':all_detail_item,'detail_result':all_detail_result}       
        item_df = pd.DataFrame.from_dict(my_dict)
        item_df['film_id'] = film_id      
        all_dfs.append(item_df)

logger.info(">>> Scraping base details")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%title%ttfc_fc_tt%'", 1000, process_details)
all_details_df = pd.concat(all_dfs)
all_details_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data/films_base_details.csv"), index=False)
all_details_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data/films_base_details.json"), orient="records")

print("ok")
del all_dfs

scan_table_limit_offset
  offset = 0
  offset = 1000
  offset = 2000
  offset = 3000
  offset = 4000
  offset = 5000
  offset = 6000
  offset = 7000
  offset = 8000
  offset = 9000
  offset = 10000
  offset = 11000
  offset = 12000
  offset = 13000
  offset = 14000
  offset = 15000
  offset = 16000
  offset = 17000
  offset = 18000
  offset = 19000
  offset = 20000
  offset = 21000
  offset = 22000
  offset = 23000
  offset = 24000
  offset = 25000
ok


Generate Awards TSV

In [ ]:
all_dfs = []

# scan pages
def process_awards(df):
    for index, row in df.iterrows():
        all_events = []
        all_winner_nominee=[]
        all_award_name=[]
        all_item=[]
        all_name=[]
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text and title elements of all reviews
        items = soup.find_all('section', class_='ipc-page-section')[:-3]
        
        if len(items) == 0:

                        
        if items:
            for item1 in items:
                h3_tags=item1.find('h3', class_='ipc-title__text')
                if h3_tags: 
                    h3_tags=item1.find('h3', class_='ipc-title__text').get_text()
                    subsection=item1.find_all('li',class_='ipc-metadata-list-summary-item')
                    if subsection:
                        for li in subsection:
                            winner_nominee = li.find('a',class_="ipc-metadata-list-summary-item__t")
                            if winner_nominee:                                    
                                winner_nominee = li.find('a',class_="ipc-metadata-list-summary-item__t").get_text()
                                award_name=li.find('span',class_='ipc-metadata-list-summary-item__tst')
                                if award_name:                                        
                                    award_name=li.find('span',class_='ipc-metadata-list-summary-item__tst').get_text()
                                    types=li.find('span',class_='ipc-metadata-list-summary-item__li')
                                    if types:
                                        types=li.find('span',class_='ipc-metadata-list-summary-item__li').get_text()
                                        allnames=li.find_all('li',class_='ipc-inline-list__item')[1:]
                                        if allnames:
                                            for allname in allnames:
                                                name=allname.find('a',class_='ipc-metadata-list-summary-item__li').get_text()
                                                all_name.append(name)
                                                all_item.append(types)
                                                all_award_name.append(award_name)
                                                all_winner_nominee.append(winner_nominee)
                                                all_events.append(h3_tags)
                                        else:
                                            name=None
                                            all_name.append(name)
                                            all_item.append(types)
                                            all_award_name.append(award_name)
                                            all_winner_nominee.append(winner_nominee)
                                            all_events.append(h3_tags)
                                    else:
                                        types=None
                                        name=None
                                        all_name.append(name)
                                        all_item.append(types)
                                        all_award_name.append(award_name)
                                        all_winner_nominee.append(winner_nominee)
                                        all_events.append(h3_tags)



        my_dict = {'event':all_events, 'winner_nominee':all_winner_nominee,'award_name':all_award_name,'item':all_item,'name':all_name}
        if len(my_dict) == 0:
            logger.debug('No awards found '+film_id)
            if "have any awards for this title yet" in html.unescape(html_content):
                logger.debug('Page content not valid '+film_id)
                my_dict = {'event':'None', 'winner_nominee':'None','award_name':'None','item':'None','name':'None'}
        
        item_df = pd.DataFrame.from_dict(my_dict)
        item_df['film_id'] = film_id      
        all_dfs.append(item_df)

logger.info(">>> Scraping Awards")
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%awards%'", 1000, process_awards)
all_awards_df = pd.concat(all_dfs)
all_awards_df.to_csv(os.path.join(os.path.dirname(__name__), "sample_data\awards.tsv"), sep='\t', index=False)
all_awards_df.to_json(os.path.join(os.path.dirname(__name__), "sample_data\awards.json"), sep='\t', index=False)

print("ok")
del all_dfs

Generate full_credits CSV

In [22]:
def clean_text(text):
    return text.strip()

# Function to extract writers and directors
def process_writers_and_directors(df):
    all_writers_directors = []
    for index, row in df.iterrows():
        html_content = decompress_string(row['page_content_zip'])
        film_id = get_film_id_from_url(row['url'])
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract writers
        writers_section = soup.find("h4", attrs={"name": "writer"})
        if writers_section:
            writers_table = writers_section.find_next("table", class_="simpleCreditsTable")
            if writers_table:
                writers = [clean_text(writer.text) for writer in writers_table.find_all("td", class_="name")]
                
                # Create DataFrame for writers
                writers_df = pd.DataFrame({"person": writers, "role": "Writer"})
                
                # Add film ID to DataFrame
                writers_df['film_id'] = film_id
                
                # Append DataFrame to list
                all_writers_directors.append(writers_df)
            else:
                print(f"Writers table not found for film ID: {film_id}")
        else:
            print(f"Writers section not found for film ID: {film_id}")
        
        # Extract directors
        directors_section = soup.find("h4", attrs={"name": "director"})
        if directors_section:
            directors_table = directors_section.find_next("table", class_="simpleCreditsTable")
            if directors_table:
                directors = [clean_text(director.text) for director in directors_table.find_all("td", class_="name")]
                
                # Create DataFrame for directors
                directors_df = pd.DataFrame({"person": directors, "role": "Director"})
                
                # Add film ID to DataFrame
                directors_df['film_id'] = film_id
                
                # Append DataFrame to list
                all_writers_directors.append(directors_df)
            else:
                print(f"Directors table not found for film ID: {film_id}")
        else:
            print(f"Directors section not found for film ID: {film_id}")
    
    # Concatenate all DataFrames
    if all_writers_directors:
        all_writers_directors_df = pd.concat(all_writers_directors)
        all_writers_directors_df.to_csv('/Users/komal/Desktop/new_logs/directors_writers.csv', index=False)
        print("Directors and Writers data saved to CSV file.")
    else:
        print("No Directors and Writers data found.")

logger.info(">>> Scraping full credits")
# Define the SQL query
scan_table_limit_offset(db_conn, "select * from pages_dump where url like '%fullcredits%'", 1000, process_writers_and_directors)
all_criticreviews_df = pd.concat(all_dfs)
all_criticreviews_df.to_csv(os.path.join(os.path.dirname(__name__), "directors_writers.csv"), sep='\t', index=False)
#print(f"Total Movies with no locations = {nolocation_counter}")
print("ok")
del all_dfs


# In[ ]:


# Read the existing CSV file into a DataFrame
existing_df = pd.read_csv('/Users/komal/Desktop/new_logs/directors_writers.csv')

# Drop any duplicate rows
existing_df = existing_df.drop_duplicates()

# Pivot the DataFrame to reshape it
pivoted_df = existing_df.pivot_table(index='film_id', columns='role', values='person', aggfunc=lambda x: ', '.join(x)).reset_index()
pivoted_df.columns.name = None  # Remove the column name for better formatting

# Print the reshaped DataFrame
pivoted_df.head(20)

pivoted_df.to_csv('/Users/komal/Desktop/new_logs/directors_writers_reshaped.csv', index=False)



OperationalError: unable to open database file